<small>This notebook was put together by [wesley beckner](http://wesleybeckner.github.io)</small>

<a id='top'></a>

# Contents

[scrape data](#scrape)

[create descriptors](#descriptors)

In [1]:
#import gains as genetic
from math import inf
import statistics
import dill
import csv
import requests
import json
import pickle
import salty
import numpy as np
from numpy import array, all
import matplotlib.pyplot as plt
import numpy.linalg as LINA
from scipy import stats
from scipy.stats import uniform as sp_rand
from scipy.stats import mode
from sklearn.linear_model import Lasso

from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
import os
import sys
import pandas as pd
from collections import OrderedDict
from numpy.random import randint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from math import log
from time import sleep
%matplotlib inline
       
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]   

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  

<a id='scrape'></a>

# Scrape ILThermo Data

[back to top](#top)

ILThermo has specific 4-letter tags for the properties in the database. These can be determined by inspecting the web elements on their website.

Melting point: prp=lcRG (note this in the paper_url string)

All that needs to be changed to scrape other property data is the 4-letter tag and the directory in which to save the information.

In [28]:
9712/30/60

5.395555555555556

In [26]:
paper_url = "http://ilthermo.boulder.nist.gov/ILT2/ilsearch?"\
    "cmp=&ncmp=1&year=&auth=&keyw=&prp=&"
#the property codes can usually be found by opening the inspection
#console and searching for "dijit dijitReset dijitInline dijitLeft 
#dijitDownArrowButton dijitSelect dijitValidationTextBox" 
#and looking at the input value  

#thermal conductivity: VvTh
#electrical conductivity: bqdZ
#viscosity: AJfy
#melting temp: lcRG
#heat capacity: bvSs

    
r = requests.get(paper_url)
print(r)
#header = r.json()['header']
papers = r.json()['res']
print(len(papers))

<Response [200]>
9712


In [25]:
paper_url = "http://ilthermo.boulder.nist.gov/ILT2/ilsearch?"\
    "cmp=&ncmp=1&year=&auth=&keyw=&prp=AJfy"
#the property codes can usually be found by opening the inspection
#console and searching for "dijit dijitReset dijitInline dijitLeft 
#dijitDownArrowButton dijitSelect dijitValidationTextBox" 
#and looking at the input value  

#thermal conductivity: VvTh
#electrical conductivity: bqdZ
#viscosity: AJfy
#melting temp: lcRG
#heat capacity: bvSs

    
r = requests.get(paper_url)
print(r)
#header = r.json()['header']
papers = r.json()['res']
i = 1
data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
print(papers)
for paper in papers[:2]:
    
    r = requests.get(data_url.format(paper_id=paper[0]))
    data = r.json()['data']
    with open("development/temp/%s.json" % i, "w") as outfile:
        json.dump(r.json(), outfile)
    #then do whatever you want to data like writing to a file
    sleep(0.5) #import step to avoid getting banned by server
    i += 1

<Response [200]>
[['TMAzg', 'Safarov et al. (2017a)', 'Viscosity', 'Liquid', 'ACISuQ', None, None, '378', '1-butyl-3-methylimidazolium tris(pentafluoroethyl)trifluorophosphate'], ['SPhJY', 'Ferreira et al. (2012a)', 'Viscosity', 'Liquid', 'ADWgol', None, None, '159', 'trihexyl(tetradecyl)phosphonium tris(pentafluoroethyl)trifluorophosphate'], ['biQXm', 'Harris and Woolf (2011)', 'Viscosity', 'Liquid', 'ABdMMy', None, None, '91', '1-butyl-1-methylpyrrolidinium bis[(trifluoromethyl)sulfonyl]imide'], ['GfuMn', 'Harris et al. (2006)', 'Viscosity', 'Liquid', 'ABCNJc', None, None, '81', '1-methyl-3-octylimidazolium tetrafluoroborate'], ['himXM', 'Harris et al. (2005)', 'Viscosity', 'Liquid', 'ABChkk', None, None, '73', '1-butyl-3-methylimidazolium hexafluorophosphate'], ['ywKLO', 'Harris and Kanakubo (2016)', 'Viscosity', 'Liquid', 'AArYxG', None, None, '65', '1-ethyl-3-methylimidazolium tetracyanoborate'], ['LULwG', 'Harris et al. (2007a)', 'Viscosity', 'Liquid', 'ABchRq', None, None, '65',

NameError: name 'json' is not defined

In [17]:
a = "fdsa"
"asdf{}".format(a)

'asdffdsa'

In [23]:
import requests
from time import sleep

def il_scrape(prp, ncmp=1):
    """
    webscraper tool for accessing backend data in the ILThermo
    browser. A few property codes are listed below, additional
    property codes can usually be found by opening the inspection
    console in your web brwoser and searching for "dijit dijitReset
    dijitInline dijitLeft dijitDownArrowButton dijitSelect
    dijitValidationTextBox" and looking at the input value in the html

    Paremeters
    ----------
    prp : str
        property ID. Current options are:
        thermal conductivity: VvTh
        electrical conductivity: bqdZ
        viscosity: blXM
        melting temp: lcRG
        heat capacity: bvSs
    ncmp : int
        number of components in IL. Default 1

    Returns
    -------
    data : json structure
    """


    paper_url = "http://ilthermo.boulder.nist.gov/ILT2/ilsearch?" \
                "cmp=&ncmp={}&year=&auth=&keyw=&prp={}".format(ncmp, prp)

    r = requests.get(paper_url)
    papers = r.json()['res']
    i = 1
    data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
    data = []
    for paper in papers[:2]:
        r = requests.get(data_url.format(paper_id=paper[0]))
        data.append(r)
        sleep(0.5)  # import step to avoid getting banned by server
        i += 1
    return data

In [24]:
il_scrape('AJfy')

[<Response [200]>, <Response [200]>]

In [9]:
ncmp='1'
prp='VvTh'

paper_url = "http://ilthermo.boulder.nist.gov/ILT2/ilsearch?" \
            "cmp=&ncmp={}&year=&auth=&keyw=&prp={}".format(ncmp, prp)

r = requests.get(paper_url)
papers = r.json()['res']
i = 1
data_url = 'http://ilthermo.boulder.nist.gov/ILT2/ilset?set={paper_id}'
data = []
for paper in papers[:]:
    r = requests.get(data_url.format(paper_id=paper[0]))
    print(r)
    data.append(r)
    sleep(0.5)  # import step to avoid getting banned by server
    i += 1


<a id='descriptors'></a>

# Create Descriptors

[back to top](#top)

The scraped data is in the form of a json file. The json files contain all the experimental information NIST has archived, including methods and experimental error!

Unfortunately the IUPAC names in the database are imperfect. We address this after the following cell.

In [4]:
###add JSON files to density.csv
outer_old = pd.DataFrame()
outer_new = pd.DataFrame()
number_of_files = 692 #set manually or use ipython magic to list dir
                      #contents

for i in range(number_of_files):
    with open("development/MELTING_POINT/%s.json" % str(i+1)) as json_file:
        
        #grab data, data headers (names), the salt name
        json_full = json.load(json_file)
        json_data = pd.DataFrame(json_full['data'])
        json_datanames = np.array(json_full['dhead'])
        json_data.columns =  json_datanames
        json_saltname = pd.DataFrame(json_full['components'])

        inner_old = pd.DataFrame()
        inner_new = pd.DataFrame()

        #loop through the columns of the data, note that some of the 
        #json files are missing pressure data. 
        if len(json_data.columns.values) > 3:
            print(json_data.columns.values)
        for indexer in range(len(json_data.columns)):
            grab=json_data.columns[indexer]
            list = json_data[grab]
            #my_list grabs first element
            #change l[0] to l[1] for error
            my_list = [l[0] for l in list]
            dfmy_list = pd.DataFrame(my_list)
            dfmy_list.columns = [json_datanames[indexer][0]]
            inner_new = pd.concat([dfmy_list, inner_old], axis=1)
            inner_old = inner_new

        #add the name of the salt    
        inner_old['salt_name']=json_saltname.iloc[0][3]           

        #add to the growing dataframe
        outer_new = pd.concat([inner_old, outer_old], axis=0)
        outer_old = outer_new
pd.DataFrame.to_csv(outer_old, path_or_buf="development/"\
        "melting_point.csv", index=False)

In [20]:
df = pd.read_csv('development/melting_point.csv',delimiter=',')
salts = pd.DataFrame(df["salt_name"])
salts = salts.rename(columns={"salt_name": "salts"})
anions= []
cations= []
missed = 0
for i in range(df.shape[0]):
    if len(salts['salts'].iloc[i].split()) == 2:
        cations.append(salts['salts'].iloc[i].split()[0])
        anions.append(salts['salts'].iloc[i].split()[1])
    elif len(salts['salts'].iloc[i].split()) == 3:
        if("sulfate" in salts['salts'].iloc[i] or\
        "phosphate" in salts['salts'].iloc[i] or\
        "phosphonate" in salts['salts'].iloc[i] or\
        "carbonate" in salts['salts'].iloc[i]):
            #these strings have two-word anions
            first = salts['salts'].iloc[i].split()[1]
            second = salts['salts'].iloc[i].split()[2]
            cations.append(salts['salts'].iloc[i].split()[0])
            anions.append(first + ' ' + second)
        elif("1:1" in salts["salts"].iloc[i]):
            #some strings contain the mixing ratio
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            cations.append(first)
            anions.append(second)
        elif("(OC-6-11)-hexafluoroantimonate(1-)" in salts["salts"].iloc[i]):
            #special case
            cations.append("1-butyl-3-methylimidazolium")
            anions.append("hexafluoroantimonate")
        elif("1,3-propanediol, 2-amino-2-(hydroxymethyl)-," in salts["salts"].iloc[i]):
            #special case
            cations.append("2-amino-2-(hydroxymethyl)-1,3-propanediol")
            anions.append("hydrochloride")
        else:
            #assume the rest are two-word cations
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            third = salts['salts'].iloc[i].split()[2]
            cations.append(first + ' ' + second)
            anions.append(third)
    elif len(salts['salts'].iloc[i].split()) == 4:
        #this particular string block contains (1:1) at end of name
        if("1,1,2,3,3,3-hexafluoro-1-propanesulfonate" in salts['salts'].iloc[i]):
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            cations.append(first + ' ' + second)
            anions.append(salts['salts'].iloc[i].split()[2])
        else:
            #and two word anion
            first = salts['salts'].iloc[i].split()[1]
            second = salts['salts'].iloc[i].split()[2]
            anions.append(first + ' ' + second)
            cations.append(salts['salts'].iloc[i].split()[0])
    elif("2-aminoethanol-2-hydroxypropanoate" in salts['salts'].iloc[i]):
        #one of the ilthermo salts is missing a space between cation/anion
        anions.append("2-hydroxypropanoate")
        cations.append("2-aminoethanol")
    elif len(salts['salts'].iloc[i].split()) == 5:
        if("bis[(trifluoromethyl)sulfonyl]imide" in salts['salts'].iloc[i]):
            anions.append("bis(trifluoromethylsulfonyl)imide")
            first = salts['salts'].iloc[i].split()[0]
            second = salts['salts'].iloc[i].split()[1]
            third = salts['salts'].iloc[i].split()[2]
            fourth = salts['salts'].iloc[i].split()[3]
            cations.append(first + ' ' + second + ' ' + third + ' ' + fourth)
        if("trifluoro(perfluoropropyl)borate" in salts['salts'].iloc[i]):
            anions.append("trifluoro(perfluoropropyl)borate")
            cations.append("N,N,N-triethyl-2-methoxyethan-1-aminium")
    elif("1H-imidazolium, 1-ethyl-3-methyl-, salt with trifluoroacetic acid (1:1)"
        in salts['salts'].iloc[i]):
        anions.append("trifluoroacetic acid")
        cations.append("1-ethyl-3-methyl-1H-imidazolium")
    elif("L-alanine, 1-methylethyl ester, dodecyl sulfate (1:1)"
        in salts['salts'].iloc[i]):
        anions.append("dodecyl sulfate")
        cations.append("L-alanine, 1-methylethyl ester")
    elif("difluorogermylene" in salts['salts'].iloc[i]):
        anions.append("difluoro")
        cations.append("germylene")
    else:
        print("uknown salt type:\t{}".format(salts['salts'].iloc[i]))
        df.drop(i, inplace=True)
        missed += 1
anions = pd.DataFrame(anions, columns=["name-anion"])
cations = pd.DataFrame(cations, columns=["name-cation"])
print("total missed salts: {}".format(missed))
print("more salts may be missing from the cation/anion info files\n\
       check final dataframe dimensions after concatenating the\n\
       cation/anion descriptors")

uknown salt type:	(t-4)-bis(2,4-pentanedionato-O,O')beryllium
uknown salt type:	2,2-dihydroxydiethylamine
uknown salt type:	1-butanaminium, N.N.N-tributyl-, salt with 2,4,6-trinitrophenol (1:1)
uknown salt type:	1-butanaminium, N.N.N-tributyl-, salt with 2,4,6-trinitrophenol (1:1)
uknown salt type:	tetrapentylammoniumbromide
total missed salts: 5
more salts may be missing from the cation/anion info files
       check final dataframe dimensions after concatonating the
       cation/anion descriptors


In [10]:
cationDescriptors = salty.load_data("cationDescriptors.csv")
cationDescriptors.columns = [str(col) + '-cation' for col in cationDescriptors.columns]
anionDescriptors = salty.load_data("anionDescriptors.csv")
anionDescriptors.columns = [str(col) + '-anion' for col in anionDescriptors.columns]

In [14]:
new_df = pd.concat([cations, anions, df["Normal melting temperature, K"]], axis=1)
new_df = pd.merge(cationDescriptors, new_df, on="name-cation", how="right")
new_df = pd.merge(anionDescriptors, new_df, on="name-anion", how="right")
new_df.dropna(inplace=True) #remove entries not in smiles database

In [33]:
pd.DataFrame.to_csv(new_df, path_or_buf='../salty/data/'\
                    'melting_point_premodel.csv', index=False)

In [31]:
new_df["Temperature, K"] = 298.15

In [32]:
new_df["Pressure, kPa"] = 101.0

In [24]:
new_df

,name-anion,smiles-anion,steiger-anion,Marsili Partial Charges-anion,BalabanJ-anion,BertzCT-anion,Ipc-anion,HallKierAlpha-anion,Kappa1-anion,Kappa2-anion,...,VSA_EState5-cation,VSA_EState6-cation,VSA_EState7-cation,VSA_EState8-cation,VSA_EState9-cation,VSA_EState10-cation,Topliss fragments-cation,"Normal melting temperature, K","Temperature, K","Pressure, kPa"
0,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,2.232639,17.934028,0.0,777.0,272.00,NaN,NaN
1,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,4.443611,15.723056,0.0,777.0,311.10,NaN,NaN
2,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,4.443611,15.723056,0.0,777.0,359.30,NaN,NaN
3,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,4.443611,15.723056,0.0,777.0,223.10,NaN,NaN
4,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,4.231806,12.768194,0.0,777.0,256.10,NaN,NaN
5,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,0.000000,20.000000,0.0,777.0,303.84,NaN,NaN
6,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,0.000000,12.000000,0.0,777.0,306.76,NaN,NaN
7,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,4.361111,22.638889,0.0,777.0,418.40,NaN,NaN
8,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,1.188333,19.311667,0.0,777.0,329.10,NaN,NaN
9,tetrafluoroborate,[B-](F)(F)(F)F,777.0,777.0,3.023716,19.119415,3.609640,-0.215065,4.784935,0.877186,...,0.0,0.0,0.0,1.232639,14.267361,0.0,777.0,459.10,NaN,NaN
